# Path API Design

>* [Calculate Path](#Calculate-Path-API)

>* [Get Path Calulation Result](#Get-Path-Calulation-Result-API)

# Calculate Path API
This API call is used to calculate the path from endpoint A (source) to endpoint B (destination). It returns the result of the calculated path in the form of a path ID (a string), and you can use the path ID in the GetPath API as the input parameter to get each hop information of the path.

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **Post** | http(s)://"IP address of NetBrain Web API Server"/ServicesAPI/API/V1/CMDB/Path/Calculation | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token* | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|sourceIP* | string  | Input the IP address of the source device.  |
|sourcePort | integer  | Specify the source protocol port If TCP/UDP is selected such as 23 for telnet. This parameter can be null.  |
|sourceGwIP* | string  | the ip address of the gateway device.  |
|sourceGwDev* | string  | the hostname of the gateway device.  |
|sourceGwIntf* | string  | the name of the gateway interface.  |
|destIP* | string  | Input the IP address of the destination device.  |
|destPort* | integer  | Specify the destination protocol port If TCP/UDP is selected, such as 23 for telnet. This parameter can be null.  |
|pathAnalysisSet* | integer  | Specify the path type to calculate:<br>▪ 1 - L3 Path<br>▪ 2 - L2 Path<br>▪ 3 - L3 Active Path |
|protocol* | integer  | Specify the application protocol. see list_of_ip_protocol_numbers, such as 4 for IPv4.  |
|isLive | bool  | ▪ false - Use data From current Baseline<br>▪ true - Use data via live access |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|taskID| string | The task ID of the calculated path. You can call the hop information of the path with the taskID in the GetPath API. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |

* **Example**

In [6]:
# import python modules 
import requests
import time
import urllib3
import pprint
import json
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set the request parameters
token = "86f3b508-4f89-4b27-8f57-33feec71a395"
nb_url = "http://192.168.28.79"
sourceIP = "10.10.3.253"
sourcePort = 0
sourceGwIP = "10.10.3.253"
sourceGwDev = "GW2Lab"
sourceGwIntf =  "GigabitEthernet0/0.10"
destIP = "172.24.32.225"
destPort = 0
pathAnalysisSet = 1
protocol = 4
isLive = 1
# Trigger path calculate and return task ID
def CalPath(nb_url, token, sourceIP, sourceGwIP, sourceGwDev, sourceGwIntf, destIP, pathAnalysisSet, protocol, isLive):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation"
   
    path_param={
            "sourceIP":sourceIP,                # IP address of the source device.
            "sourceGwDev":sourceGwDev,          # Hostname of the gateway device.
            "sourceGwIP":sourceGwIP,            # Ip address of the gateway device.
            "sourceGwIntf":sourceGwIntf,        # Name of the gateway interface.
            "destIP":destIP,                    # IP address of the destination device.  
            "pathAnalysisSet":pathAnalysisSet,  # 1:L3 Path; 2:L2 Path; 3:L3 Active Path
            "protocol":protocol,                # Specify the application protocol, check online help, such as 4 for IPv4.
            "isLive":isLive                     # False: Current Baseline; True: Live access
    } 
    try:
        #response = requests.post(full_url, data = json.dumps(path_param), headers=headers, verify=False)
        response = requests.post(full_url, data = json.dumps(path_param), headers=headers)
        if response.status_code == 200:
            result = response.json()
            #print(result)
            return result
        else:
            print(response.text)
    except Exception as e:
        return (str(e))  
    
res = CalPath(nb_url, token, sourceIP, sourceGwIP, sourceGwDev, sourceGwIntf, destIP, pathAnalysisSet, protocol, isLive)
res

{'taskID': 'b0bae173-af8c-418b-8dc4-1ffdb0e897fa',
 'statusCode': 790200,
 'statusDescription': 'Success.'}

# Get Path Calulation Result API
This API call is used to get the hop information of a path calculated through the CalcPath API. <br><br>If the Calculation Path task is not finished yet or failed,  user will get an error with messsage reminding.which means you don't need to wait anymore before trying to query the result.<br><br>All directed links in the result consists of a directed path grapth, which contains all possible reachable paths from the original source  to the destination specified in path calculation

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// http(s)://"IP address of NetBrain Web API Server"/ServicesAPI/API/V1/CMDB/Path/Calculation/{taskID}/Result | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |
|token* | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|nb_URL* | string  | IP address of your NetBrain Web API Server  |
|taskID* | string  | Input the task ID returned by the CalcPath API. |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |
|hopList| list of Object | Hop list. |
|hopList.hopId| string | Hop Guid. |
|hopList.srcDeviceName| string | Source device name. |
|hopList.inboundIneterface| string | Source interface name. |
|hopList.mediaName| string | Media name. |
|hopList.dstDeviceName| string | Destination device name. |
|hopList.outboundInterface| string | Destination interface name. |
|hopList.nextHopIdList| list of string | List of next hop ids. |


* **Example**

In [5]:
# Set the request parameters
taskID = "3402b0ad-e8ca-472f-b470-7d31bd20bcce"

# Get path result based on path task ID
def GetPathResult(nb_url, token, taskID):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    full_url= nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation/" + str(taskID) + "/Result"
    try:
        #response = requests.get(full_url, headers=headers, verify=False)
        response = requests.get(full_url, headers=headers)
        if response.status_code == 200:
            result = response.json()
            #print(result["hopList"])
            return result
        else:
            print(response.text)
    except Exception as e:
        return (str(e))   
    
res = GetPathResult(nb_url, token, taskID)
res

{"hopList":[],"statusCode":794008,"statusDescription":"Task '3402b0ad-e8ca-472f-b470-7d31bd20bcce' is failed"}
